In [1]:
import cv2
import argparse
import numpy as np

In [2]:
image = cv2.imread('dataset/08/pcd0848r.png')

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

# read class names from text file

classes = open('YoloDetection/coco.names').read().strip().split('\n')


# generate different colors for different classes
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
#net = cv2.dnn.readNet(args.weights, args.config)
net = cv2.dnn.readNetFromDarknet('YoloDetection/yolov3.cfg', 'YoloDetection/yolov3.weights')

# create input blob
blob = cv2.dnn.blobFromImage(image, scale, (Width,Height), (0,0,0), True, crop=False)

# set input blob for the network
net.setInput(blob)

In [3]:
def get_output_layers(net):

    layer_names = net.getLayerNames()

    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [4]:
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
ln

['yolo_82', 'yolo_94', 'yolo_106']

In [5]:
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
#outs = net.forward(get_output_layers(net))
outs = net.forward(ln)

# initialization
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

# for each detetion from each output layer
# get the confidence, class id, bounding box params
# and ignore weak detections (confidence < 0.5)
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [6]:

blob = cv2.dnn.blobFromImage(image, 1/255.0, (Width, Height), swapRB=True, crop=False)
#blob.shape
r = blob[0, 0, :, :]
#r.shape
cv2.imshow('blob', r)
text = f'Blob shape={blob.shape}'
cv2.displayOverlay('blob', text)
cv2.waitKey(1)

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1199: error: (-213:The function/feature is not implemented) The library is compiled without QT support in function 'cv::displayOverlay'


In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# go through the detections remaining
# after nms and draw bounding box
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]

    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

# display output image
cv2.imshow("object detection", image)

# wait until any key is pressed
cv2.waitKey()

 # save output image to disk
cv2.imwrite("object-detection.jpg", image)

# release resources
cv2.destroyAllWindows()

In [ ]:
outs